In [ ]:
# Importa bibliotecas necessárias
import cv2  # Para capturar imagens da câmera e manipulá-las.
import mediapipe as mp  # Para detectar mãos e analisar gestos.
import json  # Para salvar e carregar dados em formato JSON.
import os  # Para lidar com arquivos e diretórios no sistema operacional.

# Inicialização do módulo de detecção de mãos do MediaPipe
mp_hands = mp.solutions.hands  # Acessa a funcionalidade de detecção de mãos.
hands = mp_hands.Hands(  # Configura o detector de mãos.
    static_image_mode=False,  # Detecta mãos em vídeo em tempo real.
    max_num_hands=1,  # Limita a detecção a uma mão.
    min_detection_confidence=0.7  # Define a confiança mínima para considerar a detecção válida.
)
mp_drawing = mp.solutions.drawing_utils  # Para desenhar conexões entre os pontos da mão.

# Define nomes de arquivos e pastas
database_file = "hand_gestures.json"  # Arquivo que armazena gestos e coordenadas.
output_folder = "Coordenadas"  # Pasta onde as imagens capturadas serão salvas.

# Cria a pasta de saída, caso ela ainda não exista.
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Define as dimensões do retângulo onde a mão deve ser posicionada.
RECT_TOP_LEFT = (200, 100)  # Canto superior esquerdo do retângulo (x, y).
RECT_BOTTOM_RIGHT = (440, 400)  # Canto inferior direito do retângulo (x, y).

# Função para carregar o banco de dados
def load_database():
    if os.path.exists(database_file):  # Verifica se o arquivo de banco de dados existe.
        with open(database_file, "r") as f:  # Abre o arquivo no modo de leitura.
            return json.load(f)  # Retorna os dados como um dicionário.
    return {}  # Retorna um dicionário vazio se o arquivo não existir.

# Função para salvar dados no banco de dados
def save_database(data):
    with open(database_file, "w") as f:  # Abre o arquivo no modo de escrita.
        json.dump(data, f, indent=4)  # Salva os dados em formato JSON com indentação.

# Função para remover uma letra do banco de dados e das imagens associadas.
def delete_letter_from_database(letter):
    data = load_database()  # Carrega os dados existentes.
    if letter in data:  # Verifica se a letra está no banco de dados.
        del data[letter]  # Remove a letra do banco de dados.
        save_database(data)  # Atualiza o banco de dados.
        print(f"A letra '{letter}' foi removida do banco de dados.")

        # Remove imagens relacionadas à letra
        removed_files = 0
        for file_name in os.listdir(output_folder):  # Lista todos os arquivos na pasta de saída.
            if file_name.startswith(letter):  # Verifica se o nome do arquivo começa com a letra.
                os.remove(os.path.join(output_folder, file_name))  # Remove o arquivo.
                removed_files += 1
        print(f"{removed_files} imagens da letra '{letter}' foram removidas.")
    else:
        print(f"A letra '{letter}' não foi encontrada no banco de dados.")

# Função para salvar coordenadas no banco de dados
def save_to_database(letter, coordinates):
    data = load_database()  # Carrega o banco de dados atual.
    if letter not in data:  # Adiciona a letra se ela ainda não existir no banco.
        data[letter] = []  # Inicializa uma lista para armazenar coordenadas.
    data[letter].append(coordinates)  # Adiciona as coordenadas à lista da letra.
    save_database(data)  # Salva o banco de dados atualizado.

# Função para exibir coordenadas capturadas no console
def display_coordinates_inline(coordinates):
    coord_text = "Coordenadas capturadas: " + " ".join(  # Formata as coordenadas em uma linha.
        [f"({round(coord['x'], 2)}, {round(coord['y'], 2)})" for coord in coordinates]
    )
    print(coord_text)

# Função para verificar se a mão está completamente dentro do retângulo
def is_hand_inside_rectangle(hand_coordinates):
    for point in hand_coordinates:  # Itera sobre os pontos da mão.
        x_pixel = int(point["x"] * 640)  # Converte coordenadas relativas para pixels (assumindo largura 640).
        y_pixel = int(point["y"] * 480)  # Converte coordenadas relativas para pixels (altura 480).
        if not (RECT_TOP_LEFT[0] <= x_pixel <= RECT_BOTTOM_RIGHT[0] and
                RECT_TOP_LEFT[1] <= y_pixel <= RECT_BOTTOM_RIGHT[1]):  # Verifica se está dentro do retângulo.
            return False
    return True  # Retorna True se todos os pontos estiverem no retângulo.

# Função para capturar gestos e salvar variações para uma letra específica
def capture_variations_for_letter(letter):
    cap = cv2.VideoCapture(0)  # Inicializa a captura de vídeo da câmera.
    print(f"Iniciando captura para a letra: {letter}")
    print("Posicione a mão dentro do retângulo vermelho para capturar gestos.")
    print("Pressione 'q' para encerrar a captura.")

    img_counter = 0  # Contador para nomear as imagens.

    while cap.isOpened():  # Enquanto a câmera estiver funcionando:
        success, image = cap.read()  # Captura o quadro da câmera.
        if not success:
            print("Erro ao acessar a câmera.")  # Trata falhas na captura.
            break

        cv2.rectangle(image, RECT_TOP_LEFT, RECT_BOTTOM_RIGHT, (0, 0, 255), 2)  # Desenha o retângulo.

        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Converte o quadro para RGB.
        results = hands.process(image_rgb)  # Processa o quadro com o MediaPipe.

        message = ""  # Mensagem padrão na tela.
        if results.multi_hand_landmarks:  # Verifica se alguma mão foi detectada.
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)  # Desenha os pontos da mão.
                hand_coordinates = [{"x": landmark.x, "y": landmark.y} for landmark in hand_landmarks.landmark]

                if is_hand_inside_rectangle(hand_coordinates):  # Verifica se a mão está no retângulo.
                    save_to_database(letter, hand_coordinates)  # Salva as coordenadas.
                    display_coordinates_inline(hand_coordinates)  # Mostra as coordenadas.

                    img_name = os.path.join(output_folder, f"{letter}{img_counter}.png")  # Nome da imagem.
                    cv2.imwrite(img_name, image)  # Salva a imagem.
                    print(f"Imagem salva como: {img_name}")

                    img_counter += 1  # Incrementa o contador.
                else:
                    message = "Posicione a mão dentro do retângulo para cadastrar o gesto."
        else:
            message = "Posicione a mão dentro do retângulo para cadastrar o gesto."

        if message:
            cv2.putText(image, message, (50, 450), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2, cv2.LINE_AA)  # Mostra mensagem.

        cv2.imshow("Hand Gesture Capture", image)  # Exibe o vídeo com informações.
        if cv2.waitKey(1) & 0xFF == ord('q'):  # Encerra ao pressionar 'q'.
            break

    cap.release()  # Libera a câmera.
    cv2.destroyAllWindows()  # Fecha as janelas.
    print(f"Captura finalizada para a letra: {letter}")

# Função principal para gerenciar o cadastro de letras
def register_letters():
    while True:  # Loop até o usuário decidir não excluir mais letras.
        delete_choice = input("Quer excluir alguma letra do alfabeto? (s/n): ").strip().lower()
        if delete_choice == "s":
            letter_to_delete = input("Digite a letra que deseja excluir: ").upper()
            delete_letter_from_database(letter_to_delete)
        elif delete_choice == "n":
            break
        else:
            print("Resposta inválida. Digite 's' para sim ou 'n' para não.")
    
    letter = input("Digite a letra que deseja cadastrar: ").upper()  # Recebe a letra a ser cadastrada.
    capture_variations_for_letter(letter)  # Captura gestos para essa letra.

# Executa o cadastro de letras
register_letters()  # Inicia o fluxo principal.


Quer excluir alguma letra do alfabeto? (s/n):  n
